In [1]:
import pandas as pd 
import numpy as np
import math
import os
from glob import glob

# Creating a models of Murine V-genes

In [7]:
pwd #placed the tsv file on the desktop

'/home/kristine/Desktop'

In [8]:
murine_v = pd.read_csv('Vgene_seqs_complete_murine.tsv', sep='\t') #importing the file

In [9]:
#defining the constants

#Alpha chain
#>Alpha 
TRAV1_2_03 = "GQNIDQPTEMTATEGAIVQINCTYQTSGFNGLFWYQQHAGEAPTFLSYNVLDGLEEKGRFSSFLSRSKGYSYLLLKELQMKDSASYLCAVR"
TRAJ2_01 = "NTGGLSGKLTFGEGTQVTVIS"

#Beta chain
#>Beta
TRBD1 = "GTGG" 
TRBJ1_3_01= "SGNTLYFGEGSRLIVV"

In [10]:
#Creating all the fasta files. Also on my desktop

for i in range(len(murine_v)):
    file1 = open(murine_v.gene[i]+".fasta","w") 
    alpha = [">alpha \n", TRAV1_2_03+TRAJ2_01]
    space = ["\n"]
    beta = [">beta \n", murine_v.seq[i]+TRBD1+TRBJ1_3_01]  
    file1.writelines(alpha)
    file1.writelines(space)
    file1.writelines(beta)

    file1.close()

In [71]:
#moved all the fasta files into a folder on the desktop names "fasta"

In [78]:
#now i need to figure out/remember how to use LYRA in the terminal. 

    # command : conda env list
# lyra runs in python 2.7 - i have this installed in my python 2 envionment
    # command: source activate python2_etc

    
#lyra_model ./TRBV1*01.fasta -o TRBV1*01.pdb

#Error: File format problem in trying to open HMM file ./TRBV12-1*02.fasta.
#Format tag is '>alpha': unrecognized.
#Current H3 format is 'HMMER3/f'. Previous H2/H3 formats also supported.


In [105]:
#### it is not working #### 
#I do not know how to trouble shoot - this might be due to the fact that i did not install it myself 
#in the first place, that was done by Magnus. Therefore i am using the online tool



#the online tool is used and all but TRBV30 and 31*1 and 31*2 was modelled. 
#they were trimmed using Milenas Jupyter notebook. 


#now to the calculation of the RMSD:

# calculating the RMSD of the murine V-genes

In [11]:
# ---------------------------------- IMPORT OF PACKAGES -----------------------
from Bio.PDB import *
import numpy as np
from Bio.SVDSuperimposer import SVDSuperimposer
import os
import pandas as pd
from glob import glob

# ---------------------------------- REFERENCES -------------------------------
#Bio.PDB:
#Hamelryck, T., Manderick, B. (2003) PDB parser and structure class implemented in Python. Bioinformatics 19: 2308–2310

#SVD superimposer:
#Matrix computations, 2nd ed. Golub, G. & Van Loan, CF., The Johns Hopkins University Press, Baltimore, 1989

#http://pldserver1.biochem.queensu.ca/~rlc/work/pymol/

#LYRA, SCRWL, SKLEARN, HMMER, Random forest
# --------------------------------------------------------------------------------

#choose some true structures, PDB file format
parser = PDBParser()

def pdb_load(pdb_files):
    """a function that converts a pdb path into a strcutural file"""
    parser = PDBParser()
    
    structure_list = []
    
    for fileName in pdb_files:
        #the fileName is the whole path
        #structre_id is the created based on the path, cutting it at the last "/" and removing the last letters "_CDRX.pdb"
        structure_id = fileName.rsplit('/', 1)[1][:-9]
        #the stucture is imported
        structure = parser.get_structure(structure_id, fileName)
        
        structure_list.append(structure)
    
    return structure_list

def get_CA_coordinates(structure):
    """this function return a vector of CA atom coordinates"""
    #empty list
    x = []
    
    for model in structure.get_list():
        for chain in model.get_list():
            for residue in chain.get_list():
                if residue.has_id("CA"):
                    ca = residue["CA"]
                    a = ca.get_coord()
                    
                    x.append(a)
                    
    coordinates = np.array(x)
    return coordinates

def get_coordinates(structure_list):
    """a function that allow coordinate retraction of an entire list"""
    coordinate_list = []
    
    for ii in structure_list:
        coordinates = get_CA_coordinates(ii)
        
        coordinate_list.append(coordinates)
    return coordinate_list

def get_RMSD_vers2(structure1_coordinates, structure2_coordinates):
    """a function that calculate RMSD of two stuctures"""
    sup = SVDSuperimposer()
    
    if len(structure1_coordinates) != len(structure2_coordinates):
        
        if len(structure1_coordinates) < len(structure2_coordinates):
            
            length = len(structure1_coordinates)
 
            #creates two empy matrixes that is the length of the fixed. 

            structure1_coordinates_coord = np.zeros((length, 3))
            structure2_coordinates_coord = np.zeros((length, 3))
        
            for i in range(0, length): 
                structure1_coordinates_coord[i] = structure1_coordinates[i]
                structure2_coordinates_coord[i] = structure2_coordinates[i]

            sup.set(structure1_coordinates_coord, structure2_coordinates_coord)
            sup.run()
            rms = sup.get_rms()
            return("%.2f"%rms)
        
        elif len(structure2_coordinates) < len(structure1_coordinates):
            
            length = len(structure2_coordinates)
            #creates two empy matrixes that is the length of the fixed. 

            structure2_coordinates_coord = np.zeros((length, 3))
            structure1_coordinates_coord = np.zeros((length, 3))
        
            for i in range(0, length): 
                structure2_coordinates_coord[i] = structure2_coordinates[i]
                structure1_coordinates_coord[i] = structure1_coordinates[i]

            
            sup.set(structure2_coordinates_coord, structure1_coordinates_coord)
            sup.run()
            rms = sup.get_rms()
            return("%.2f"%rms)

    else:
        sup.set(structure1_coordinates,structure2_coordinates)
        sup.run()
        rmsd = sup.get_rms()
        return ("%.2f"%rmsd)

### get them into a matrix
        
def RMSD_metric(x,y,names):
    """a function that allow RMSD to be calculated on many"""
    matrix = []
    
    for i in x:
        for j in y:
            t = get_RMSD_vers2(i,j)
            matrix.append(t)
            
    matrix = np.array(matrix)
    matrix = np.reshape(matrix, (len(names), len(names)))

    #naming the columns and rows, and converting it to a data-frame. 
    df = pd.DataFrame(matrix)
    df.columns = (names)
    df.index = (names)
    return df 


def run_canonical_rmsd(pdb_files):
    """ a function that pull data through all the other functions"""
    names = []
    for ii in pdb_files:
        id = ii.rsplit('/', 1)[1][:-9]
        names.append(id)
    
    x = pdb_load(pdb_files) 
    x_coordinates = get_coordinates(x)
    RMSD = RMSD_metric(x_coordinates, x_coordinates, names)

    return RMSD

In [7]:
#### running it ####
def RMSD_dist_file(path_to_pdb_files, cdr_n):
    """path to pdb files to include last /.
    
    cdr_n to be either 'CDR1' or 'CDR2'
    """
    cdr_n_files = glob(path_to_pdb_files +'*'+cdr_n+'.pdb')
    
    cdr_n_RMSD = run_canonical_rmsd(cdr_n_files)
    cdr_n_max_rmsd = cdr_n_RMSD.max().max()
    
    alpha = cdr_n_RMSD.to_numpy()
    alpha = alpha.astype(float)
    alpha = np.divide(alpha, cdr_n_max_rmsd) 
    cdr_n_RMSD = pd.DataFrame(alpha,cdr_n_RMSD.index,cdr_n_RMSD.columns)
    
    return cdr_n_RMSD

CDR1_RMSD = RMSD_dist_file('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/','CDR1')
CDR1_RMSD.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR1_RMSD.csv', index = True)

CDR2_RMSD = RMSD_dist_file('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/','CDR2')
CDR2_RMSD.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR2_RMSD.csv', index = True)

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarnin

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarnin

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element ''
  warnings.warn(msg, PDBConstructionWarn

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element ''
  warnings.warn(msg, PDBConstructionWarn

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=NZ) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'O' for Atom (name=OD1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'O' for Atom (name=OD2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionW

# Calculating the TM Score


### First problem: Both the TMscore and TMalign algorithm takes offense to three of the models: 
    #### TRBV12-2*01_CDR2.pdb
    #### TRBV12-2*02_CDR2.pdb
    #### TRBV21*01_CDR2.pdb

### Second problem: The matrix is not symmetric, due to the normalization of length based on the "template"

In [12]:
#This is code from before i realised the second problem


#------------------ step 1: getting all the files imported

CDR1_files = glob('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/*CDR1.pdb')
CDR2_files = glob('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/*CDR2.pdb')

#print("./TMscore "+cs1_pdb_files[0]+" "+cs1_pdb_files[1])
# i have an empty file called terminal_output.txt ready to print in


#------------------ step 2: create a bash script to run all trough here TMscore

def doc1(liste,new,n):
    x = []
    for i in range(len(liste)):
        for j in range(len(new)):
            y = "./TMscore "+new[j]+" "+liste[i] + " >> terminal_output_CDR%s.txt" %n
            u = "echo %s,%s >> terminal_output_CDR%s.txt \n" % (i,j,n)
            x.append(y)
            x.append(u)
        #y = "./TMscore /home/kristine/Documents/Thesis/Thesis_Ubuntu/Canonical_structures/CDR1/TRBV5-6_01_CDR1.pdb "+liste[i]    
        #x.append(y)
    print(len(x))
    return x

#TM_CDR1 = doc1(CDR1_files, CDR1_files, 1)
#TM_CDR2 = doc1(CDR2_files, CDR2_files, 2)
#output:./TMscore /home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/TRBV3*02_CDR1.pdb /home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/TRBV3*02_CDR1.pdb

#type(TM_CDR1)
with open('tm_commands_CDR1.txt', 'w') as f:
    for item in TM_CDR1:
        f.write("%s\n" % item)

with open('tm_commands_CDR2.txt', 'w') as f:
    for item in TM_CDR2:
        f.write("%s\n" % item)

#------------------ step 3: rename the .txt to .sh and run it in shell
        
#./tm_commands_CDR1.sh
#./tm_commands_CDR2.sh

NameError: name 'TM_CDR1' is not defined

In [13]:
#------------------ step 4: Get a list of names for the structures (V gene names)

CDR1_names = []
for i in range(len(CDR1_files)):
    name = CDR1_files[i].rsplit('/', 1)[1][:-9]
    CDR1_names.append(name)

    
CDR2_names = []
for i in range(len(CDR2_files)):
    name = CDR2_files[i].rsplit('/', 1)[1][:-9]
    CDR2_names.append(name)
    
#----------------One time fix to remove the offending names

#CDR1_names.remove("TRBV12-2*01")
#CDR1_names.remove("TRBV12-2*02")
#CDR1_names.remove("TRBV21*01")

#CDR2_names.remove("TRBV12-2*01")
#CDR2_names.remove("TRBV12-2*02")
#CDR2_names.remove("TRBV21*01")

In [14]:
#------------------ New approach: use TMalign 

def lines_that_contain(string, fp):
    return [line for line in fp if string in line]

def getting_tm(textfile):
    with open(textfile, "r") as fp:
        score = []        
        for line in lines_that_contain("length of Chain_1", fp):
            a = float(line[10:16]) 
            score.append(a)
    return score

    #comes out in the same ordxer. 

    
#------------------ Create Matrix and place it in a pandas dataframe     
def reshape(path):   
    list_of_TM = getting_tm(path)
    
    b = list_of_TM
    b = np.array(b)
    b = 1-b
    b = np.reshape(b, (len(CDR1_names),len(CDR1_names))) #should be 46
    cdr_TM = pd.DataFrame(b, CDR1_names, CDR1_names)
    
    return cdr_TM
                   
#------------------ Print it into a CSV                   
#CDR1_TM = reshape("/home/kristine/Downloads/terminal_output_CDR1.txt")
#CDR1_TM.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR1_TM.csv', index = True)

#CDR2_TM = reshape("/home/kristine/Downloads/terminal_output_CDR2.txt")
#CDR2_TM.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR2_TM.csv', index = True)



In [15]:
#Check if symmetric

arr = CDR1_TM.to_numpy()
(arr.transpose() == arr).all()

#false

NameError: name 'CDR1_TM' is not defined

# I need to re-calculate the human genes to see, if the problem in fact is the three stupid genes

In [122]:
CDR1_human_files = glob('/home/kristine/Documents/Thesis/Thesis_Ubuntu/Canonical_structures/CDR1/all/*.pdb')
CDR2_human_files = glob('/home/kristine/Documents/Thesis/Thesis_Ubuntu/Canonical_structures/CDR2/all/*.pdb')

def doc2(liste,new,n):
    x = []
    for i in range(len(liste)):
        for j in range(len(new)):
            y = "./TMalign "+new[j]+" "+liste[i] + " >> human_terminal_output_CDR%s.txt" %n
            u = "echo %s,%s >> human_terminal_output_CDR%s.txt \n" % (i,j,n)
            x.append(y)
            x.append(u)
    print(len(x))
    return x
#In shell: 
#./human_tm_commands_CDR1.sh
#./human_tm_commands_CDR2.sh

In [123]:
TM_human_CDR1 = doc2(CDR1_human_files, CDR1_human_files, 1)
TM_human_CDR2 = doc2(CDR2_human_files, CDR2_human_files, 2)

32768
32768


In [124]:
with open('human_tm_commands_CDR1.txt', 'w') as f:
    for item in TM_human_CDR1:
        f.write("%s\n" % item)

with open('human_tm_commands_CDR2.txt', 'w') as f:
    for item in TM_human_CDR2:
        f.write("%s\n" % item)

In [125]:
CDR1_human_names = []
for i in range(len(CDR1_human_files)):
    name = CDR1_human_files[i].rsplit('/', 1)[1][:-4]
    CDR1_human_names.append(name)
    
CDR2_human_names = []
for i in range(len(CDR2_human_files)):
    name = CDR2_human_files[i].rsplit('/', 1)[1][:-4]
    CDR2_human_names.append(name)

# The code for finding the correct TM score

In [101]:
#---------------- step 1: make a descision: It was decided to always use the longest structure as the template

#---------------- step 2: know the output:

#Name of Chain_1: /home/kristine/Documents/Thesis/Thesis_Ubuntu/Canonical_structures/CDR1/all/TRBV7-2_04.pdb (to be superimposed onto Chain_2)
#Name of Chain_2: /home/kristine/Documents/Thesis/Thesis_Ubuntu/Canonical_structures/CDR1/all/TRBV7-2_04.pdb
#Length of Chain_1: 7 residues
#Length of Chain_2: 7 residues

#Aligned length= 7, RMSD=   0.00, Seq_ID=n_identical/n_aligned= 1.000
#TM-score= 1.00000 (if normalized by length of Chain_1, i.e., LN=7, d0=0.50)
#TM-score= 1.00000 (if normalized by length of Chain_2, i.e., LN=7, d0=0.50)
#(You should use TM-score normalized by length of the reference structure)

#(":" denotes residue pairs of d <  5.0 Angstrom, "." denotes other aligned residues)
#ISGHTAL
#:::::::
#ISGHTAL

In [154]:
#def lines_that_contain(string, fp):
#    return [line for line in fp if string in line]

def getting_correct_tm(textfile):
    with open(textfile, "r") as fp:
        lines = []        
        for line in lines_that_contain("TM-score= ", fp):
            a = line
            TM = a[10:-59]
            TM = float(TM)
            
            lines.append(TM)
            
            LN = a[64:-11]
            LN = float(LN)
            
            lines.append(LN)  
            
        lines = np.array(lines)
        lines = np.reshape(lines, (-1,4))
        
    return lines

def tm_def(array, name_list):
    correct_TM_list = []
        
    for i in range(len(array)):
        if array[i][3] == array[i][1]:
            correct_TM = array[i][0]
            
            correct_TM_list.append(correct_TM)
        
        elif array[i][1] > array[i][3]:
            correct_TM = array[i][0]
            
            correct_TM_list.append(correct_TM)
            
        elif array[i][3] > array[i][1]:
            correct_TM = array[i][2]
            
            correct_TM_list.append(correct_TM)
    
    correct_TM_list = np.array(correct_TM_list)
    correct_TM_list = 1-correct_TM_list
    
    correct_TM_matrix = np.reshape(correct_TM_list, (len(name_list),len(name_list)))
    cdr_TM = pd.DataFrame(correct_TM_matrix, name_list, name_list)
    
    return cdr_TM

In [132]:
#human_TM_CDR1 = getting_correct_tm("/home/kristine/Downloads/human_terminal_output_CDR1.txt")
#human_TM_CDR1 = tm_def(human_TM_CDR1, CDR1_human_names)

In [133]:
#--------------- CHECK FOR SYMMETRY

arr = human_TM_CDR1.to_numpy()
(arr.transpose() == arr).all()

True

In [134]:
#------------- Save as a CSV file
human_TM_CDR1.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/human_CDR1_TM.csv', index = True)

#### Hence the CDR1 file is correct

         However, something goes wrong with CDR2 - here symmetry is not there

In [180]:
human_TM_CDR2 = getting_correct_tm("/home/kristine/Downloads/human_terminal_output_CDR2.txt")
human_TM_CDR2 = tm_def(human_TM_CDR2, CDR2_human_names)

#arr = human_TM_CDR2.to_numpy()
#(arr.transpose() == arr).all()
#false

human_TM_CDR2.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/human_CDR2_TM.csv', index = True)

In [172]:

only_once = human_TM_CDR2.drop_duplicates(keep=False)

only_once

,TRBV7-2_04,TRBV20-1_06,TRBV7-9_02,TRBV5-6_01_CDR2,TRBV24-1_01,TRBV29_OR9-2_01_CDR2,TRBV12-4_02,TRBV7-6_02,TRBV20-1_03,TRBV4-1_02,...,TRBV4-2_01,TRBV4-3_04,TRBV10-3_02,TRBV11-3_03,TRBV7-8_02,TRBV15_03,TRBV9_02,TRBV3-1_01,TRBV5-4_04,TRBV7-1_01
TRBV5-6_01_CDR2,0.42898,0.68795,0.42441,0.00000,0.55429,0.68784,0.31726,0.43375,0.68795,0.35143,...,0.35138,0.35138,0.55429,0.53310,0.35522,0.40993,0.34934,0.35143,0.40994,0.42916
TRBV6-8_01,0.56941,0.62537,0.57483,0.55827,0.23226,0.62521,0.68020,0.56930,0.62537,0.64324,...,0.64326,0.64326,0.23226,0.61242,0.54224,0.67042,0.61260,0.64330,0.67049,0.56930
TRBV25-1_01,0.51964,0.81013,0.58168,0.71024,0.25856,0.81003,0.64836,0.51960,0.81013,0.73288,...,0.73287,0.73287,0.25856,0.60528,0.53996,0.73115,0.70462,0.73293,0.73123,0.51960
TRBV21_OR9-2_01,0.25764,0.60339,0.26578,0.38590,0.53260,0.60314,0.29959,0.26302,0.60339,0.35419,...,0.35401,0.35401,0.53260,0.14194,0.30597,0.45804,0.20262,0.35415,0.45778,0.25780
TRBV15_01,0.51891,0.62934,0.49541,0.68790,0.14207,0.62913,0.62063,0.51885,0.62934,0.62745,...,0.62740,0.62740,0.14207,0.52356,0.46749,0.67441,0.57428,0.62751,0.67446,0.51885
TRBV11-2_03,0.27721,0.58946,0.30309,0.45598,0.55621,0.58923,0.45830,0.17402,0.58946,0.47915,...,0.47903,0.47903,0.55621,0.12500,0.27090,0.53310,0.36915,0.47910,0.53303,0.27727
TRBV6-1_01,0.47929,0.53543,0.53893,0.67186,0.23563,0.53540,0.62268,0.47927,0.53543,0.58085,...,0.58080,0.58080,0.23563,0.51713,0.51274,0.65398,0.56035,0.58090,0.65406,0.47927
TRBV29-1_03_CDR2,0.71433,0.00000,0.71879,0.68785,0.64345,0.00000,0.66569,0.71708,0.00000,0.53432,...,0.53426,0.53426,0.64345,0.71924,0.75256,0.57749,0.54557,0.53438,0.57740,0.71441
TRBV28_01,0.54766,0.57759,0.59390,0.52091,0.32858,0.57733,0.67961,0.54773,0.57759,0.63219,...,0.63218,0.63218,0.32858,0.58442,0.52623,0.68202,0.60963,0.63223,0.68210,0.54773
TRBV27_01,0.53646,0.62169,0.54480,0.65993,0.19959,0.62158,0.62299,0.53649,0.62169,0.55807,...,0.55811,0.55811,0.19959,0.54651,0.51684,0.63643,0.56487,0.55810,0.63656,0.53649


# Try the logic on the Murine CDR1 and CDR2

In [173]:
murine_CDR1_files = glob('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/*CDR1.pdb')
murine_CDR2_files = glob('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/*CDR2.pdb')

murine_CDR1_names = []
for i in range(len(murine_CDR1_files)):
    name = murine_CDR1_files[i].rsplit('/', 1)[1][:-9]
    murine_CDR1_names.append(name)

    
murine_CDR2_names = []
for i in range(len(murine_CDR2_files)):
    name = murine_CDR2_files[i].rsplit('/', 1)[1][:-9]
    murine_CDR2_names.append(name)
    
#----------------One time fix to remove the offending names

murine_CDR1_names.remove("TRBV12-2*01")
murine_CDR1_names.remove("TRBV12-2*02")
murine_CDR1_names.remove("TRBV21*01")

murine_CDR2_names.remove("TRBV12-2*01")
murine_CDR2_names.remove("TRBV12-2*02")
murine_CDR2_names.remove("TRBV21*01")

In [174]:
murine_TM_CDR1 = getting_correct_tm("/home/kristine/Downloads/murine_terminal_output_CDR1.txt")
murine_TM_CDR1 = tm_def(murine_TM_CDR1, murine_CDR1_names)

murine_TM_CDR2 = getting_correct_tm("/home/kristine/Downloads/murine_terminal_output_CDR2.txt")
murine_TM_CDR2 = tm_def(murine_TM_CDR2, murine_CDR2_names)

In [175]:
#--------------- CHECK FOR SYMMETRY

arr = murine_TM_CDR1.to_numpy()
(arr.transpose() == arr).all()

True

In [177]:
#--------------- CHECK FOR SYMMETRY

arr = murine_TM_CDR2.to_numpy()
(arr.transpose() == arr).all()

True

In [178]:
murine_TM_CDR1.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/murine_CDR1_TM.csv', index = True)

In [179]:
murine_TM_CDR2.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/murine_CDR2_TM.csv', index = True)

# fixing the human CDR2
### notice that i had to change computer (a windows with nothing installed)

    1) i have found all the numers that were not symmetrical using if statements in excel 
    (Each place the lower triangle of the matrix and upper triangle does not match it is colored red)
    
    2) then i have manually run these and replaced the number (22 places)

    3) i saved this as a CSV file and import below to test the symmetry

In [2]:
pwd

'C:\\Users\\KDE\\Desktop\\thesis'

In [3]:
human_CDR2_sym = pd.read_csv('human_CDR2_symmetrical.csv') #importing the file

In [6]:
human_CDR2_sym = human_CDR2_sym.set_index("Unnamed: 0")

In [7]:
arr = human_CDR2_sym.to_numpy()
(arr.transpose() == arr).all()

True

In [8]:
#testing the CDR1 for the murine
murine_CDR1_sym = pd.read_csv('murine_CDR1_symmetrical.csv') #importing the file
murine_CDR1_sym = murine_CDR1_sym.set_index("Unnamed: 0")

arr = murine_CDR1_sym.to_numpy()
(arr.transpose() == arr).all()

True

In [9]:
#testing the CDR2 for the murine
murine_CDR2_sym = pd.read_csv('murine_CDR2_symmetrical.csv') #importing the file
murine_CDR2_sym = murine_CDR2_sym.set_index("Unnamed: 0")

arr = murine_CDR2_sym.to_numpy()
(arr.transpose() == arr).all()

True